# Formalities #



## Imports

In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from timeit import default_timer as timer
from numba import jit, njit, vectorize

## Custom Plot Colormap ##

In [2]:
plt.style.use('seaborn-white')
class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

## Trig Functions in Degrees



``` 
math.sin(angle*math.pi/180)
math.cos(angle*math.pi/180)
math.tan(angle*math.pi/180)
```



# Velocity and Angle Effect on Height of Ball at Target Distance

##Constants##


In [47]:
Ub = 5 # Upper Bound of Target
Lb = 4 # Lower Bound of Target
Ht = 4.5 # Midpoint of Target
Hs = 0.5 # Height of Shooter
Dt = 3 # Distance to Target
Db = 0.18 # Diameter of Ball
halfGrav = -9.81/2

In [48]:
angle=np.arange(1, 90, 0.1)
velocity=np.arange(0.01, 25.01, 0.01)
X, Y = np.meshgrid(angle, velocity)

## Function ##

In [49]:
@vectorize
def findHeightAtTarget(angle, velocity):
  t = Dt/(velocity*math.cos(angle*math.pi/180))
  return (halfGrav*(t**2))+(velocity*math.sin(angle*math.pi/180)*t)+(Hs)

In [50]:
@vectorize
def vecCondition(item):
  u = -Db/2
  b = Db/2
  if item<b and item>u:
      return 0
  return 1

In [51]:
findHeightAtTarget(45, 10)

2.6170999999999998

## Plotting

In [52]:
Z = vecCondition(findHeightAtTarget(X, Y)-Dt)
%matplotlib widget
plt.contourf(X, Y, Z, 50, norm=MidpointNormalize(midpoint=0.), cmap='seismic')
plt.colorbar();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Testing

## Speed of Function ##

In [8]:
def std(aList, bList):
  result = np.zeros(shape=(len(aList),len(bList)))
  for a in range(len(aList)):
    for b in range(len(bList)):
      result[a][b]=findHeightAtTarget(aList[a], bList[b])
  return result

In [14]:
array1=np.arange(1.0, 90.1, 0.1)
array2=np.arange(0.001, 90.001, 0.001)
A, B = np.meshgrid(angle, velocity)

In [15]:
%time findHeightAtTarget(A, B)

Wall time: 499 ms


array([[-4.41583948e+05, -4.41612195e+05, -4.41643135e+05, ...,
        -3.62300179e+10, -1.44919632e+11, -1.98022905e+35],
       [-1.10395573e+05, -1.10402631e+05, -1.10410362e+05, ...,
        -9.05750382e+09, -3.62299066e+10, -4.95057261e+34],
       [-4.90643922e+04, -4.90675260e+04, -4.90709592e+04, ...,
        -4.02555678e+09, -1.61021798e+10, -2.20025450e+34],
       ...,
       [ 5.51658547e-01,  5.56896249e-01,  5.62134299e-01, ...,
         8.01955896e+02,  1.48746312e+03, -3.16887347e+26],
       [ 5.51658603e-01,  5.56896306e-01,  5.62134355e-01, ...,
         8.01960534e+02,  1.48748168e+03, -3.16861996e+26],
       [ 5.51658660e-01,  5.56896362e-01,  5.62134412e-01, ...,
         8.01965172e+02,  1.48750023e+03, -3.16836647e+26]])

In [16]:
%time std(array1, array2)

Wall time: 4min 31s


array([[-4.41584495e+07, -1.10396120e+07, -4.90649390e+06, ...,
         5.46913292e-01,  5.46913413e-01,  5.46913534e-01],
       [-4.41612747e+07, -1.10403183e+07, -4.90680781e+06, ...,
         5.52150691e-01,  5.52150812e-01,  5.52150933e-01],
       [-4.41643693e+07, -1.10410919e+07, -4.90715164e+06, ...,
         5.57388408e-01,  5.57388529e-01,  5.57388650e-01],
       ...,
       [-3.62300187e+12, -9.05750467e+11, -4.02555763e+11, ...,
         4.12629140e+02,  4.12639080e+02,  4.12649020e+02],
       [-1.44919633e+13, -3.62299084e+12, -1.61021815e+12, ...,
        -6.98391570e+01, -6.97993965e+01, -6.97596374e+01],
       [-1.98022905e+37, -4.95057261e+36, -2.20025450e+36, ...,
        -2.44483588e+27, -2.44478155e+27, -2.44472722e+27]])

## Plotting

In [7]:
a=np.arange(25, 90.01, 0.1, dtype="float32")
b=np.arange(7.5, 25.1, 0.01, dtype="float32")
X, Y = np.meshgrid(a, b)
Z = vecCondition(findHeightAtTarget(X, Y)-Dt)
#Z = findHeightAtTarget(X, Y)-Dt
%matplotlib widget 
plt.contourf(X, Y, Z, 50, norm=MidpointNormalize(midpoint=0.), cmap='seismic')
plt.show()
plt.colorbar()
#print(X)
#print(Y)
#print(Z)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Speed of Condiontal Formatting ##

In [30]:
def normalCondition(array):
  u = -0.5
  b = 0.5
  for i in range(len(array)):
    if array[i]<b and array[i]>u:
      array[i]=0
      
test = np.linspace(-1, 1, 20000001, dtype="float32")

In [23]:
%time normalCondition(test)

Wall time: 6.56 s


In [35]:
%time vecCondition(test)

Wall time: 85.7 ms


array([-1.        , -0.99999988, -0.99999982, ...,  0.99999982,
        0.99999988,  1.        ])